# Sources
* https://medium.com/@luca.pasquarelli.villa/spotify-api-get-your-liked-songs-with-python-and-spotipy-175c2310f0c3

# Init

In [13]:
import spotipy, os

from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

os.environ["SPOTIPY_CLIENT_ID"    ] = CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = CLIENT_SECRET

In [ ]:
auth_manager = SpotifyClientCredentials()
spot         = spotipy.Spotify( auth_manager = auth_manager )

# Playlist Items

In [6]:
from pprint import pprint

# https://open.spotify.com/playlist/02iS5AFGp8YVuUUqcQf8ys?si=566c3040aa6d480b

_RESPONSE_LIMIT = 100
playlistID      = "02iS5AFGp8YVuUUqcQf8ys"


response = spot.user_playlist_tracks(
    CLIENT_ID, 
    playlistID, 
    fields = 'items,uri,name,id,total', 
    limit  = _RESPONSE_LIMIT
)

trackCount = 0
Ntracks    = response['total']
plTracks   = []

print( f"There are {Ntracks} items in this playlist ...\n" )



while 1:

    trackCount += len(response['items'])
    plTracks.extend( response['items'] )
    print( f"\nThe response has {len(response['items'])} elements. {trackCount} total tracks collected ...\n" )
        
    for item in response['items']:
        track = item['track']
        # pprint( track )
        # print()
        artists = []
        for artist in track['artists']:
            if 'name' in artist:
                artists.append( artist['name'] )
        title = track['name']
        print( f"\"{title}\" by {artists[0] if (len(artists)==1) else artists}" )

    if trackCount >= Ntracks:
        break

    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlistID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT,
        offset = trackCount
    )

print( f"\n{len(plTracks)} total tracks collected!" )

There are 341 items in this playlist ...


The response has 100 elements. 100 total tracks collected ...

"Zagros" by Strunz & Farah
"Mary Always" by Khruangbin
"Mountain" by Joachim Pastor
"The Heights" by ['Krosia', 'Alpha Room']
"Separation" by Camel Driver
"Zionsville" by Khruangbin
"Grand Artica" by Chapelier Fou
"Feud: Bette and Joan - Epilogue" by Mac Quayle
"Haw Phin" by Yin Yin
"I Want to Act" by Mac Quayle
"Fortunes" by Home
"Donatella's Spotlight" by Mac Quayle
"Algetran" by ['Sound Quelle', 'Jackarta']
"1.5_4-issacsbestmove.bwf" by Mac Quayle
"Cider Time" by Lifeformed
"405.3 Thumb CAD" by Mac Quayle
"Accelerated" by Miami Nights 1984
"Another Stage" by Mac Quayle
"Hawaii Oslo" by Hania Rani
"In Motion" by Memorex Memories
"Djinn" by Abanamat
"Cote D'Azur" by West Siberian Jazz Quartet
"Stella By Starlight" by West Siberian Jazz Quartet
"Sweet Georgia Brown" by West Siberian Jazz Quartet
"Song of Love" by West Siberian Jazz Quartet
"Days of Wine and Roses" by West Siberian 

# Reorder Playlist Tracks

In [14]:
import spotipy
import spotipy.util as util
from IPython.display import clear_output

token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

spot = spotipy.Spotify( auth = token )
clear_output( wait = False )

In [16]:
from math import ceil
from random import randrange

chunkSize = 100
Nchunks   = int( ceil( Ntracks / chunkSize ) )
Nmoves    = int( Ntracks / chunkSize )

for _ in range( Nmoves ):
    bgnDex   = randrange(0,Nchunks) * chunkSize
    moveSize = min( chunkSize, Ntracks - bgnDex )
    response = spot.playlist_reorder_items(
        playlistID, 
        bgnDex, 
        randrange(0,Ntracks), 
        range_length=chunkSize
    )
    print( response, '\n' )

{'snapshot_id': 'NzU3LDY3NWNjOGM4YmMyZGExZWQ5YzQwYmE5MDc2Y2ZlZmYzMjYxYTc5OTA='} 

{'snapshot_id': 'NzU4LGFjMDNkOTAwMDFiYTdjZjM1MTgwZGMxZWJlZTgwOTJhYWU1YzQ5NTk='} 

{'snapshot_id': 'NzU5LGVkZjRjMWEwYzE4N2JlNjdhZDRiM2Q3NmViMzY0MzY3MTUxYWI0M2M='} 



In [19]:
def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Nchunks  = int( ceil( Ntracks / chunkSize ) )
    bgnDex   = randrange(0,Nchunks) * chunkSize
    moveSize = min( chunkSize, Ntracks - bgnDex )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for _ in range( Nmoves ):
        response = spot.playlist_reorder_items(
            playlist_ID, 
            bgnDex, 
            randrange(0,Ntracks), 
            range_length = chunkSize
        )
        if verbose:
            print( response )

In [20]:
reorder_playlist_by_chunks( playlistID, 10, verbose = 1 )

{'snapshot_id': 'NzYwLDFjMjk3ZmQ5NzkyZWUxMjU4M2RhYTVmZjY2ZjZiOTdhNDliZGEzYTY='}
{'snapshot_id': 'NzYxLDRmOWRlZDM0ODQ4ZTRmZmIyZWJhODMxMGM2M2Q5YjQ5YTgxYzNjOGY='}
{'snapshot_id': 'NzYyLDE0MDkwYTIxZTk3YjBhZjcyM2E2ZTYyNmU4M2JmOTkxOTg2ZGVlOGM='}
{'snapshot_id': 'NzYzLGEyMTA0YzlmZDE1ZTdlNWNkZWNjMDliODVjNzQwNThlMmY1NDU2NjU='}
{'snapshot_id': 'NzY0LGNhMzZlNTg0YzZhYTRlMTljYmM3ZDJiMTEyZDIyYTgwYTRkOWY1NTQ='}
{'snapshot_id': 'NzY1LDk4N2NhMmQwYzY5MzBiOTlhMDcyZjA3ZWY2ZjQxNWRlZWRjZmJmNTY='}
{'snapshot_id': 'NzY2LDYzM2Q2ZmZlYWVjMGFjMTA1MmViZTAyMWM4MDc3MGZmMTkxMmEwOTY='}
{'snapshot_id': 'NzY3LGUwZDU4YjU4NDhiZDA0OWNlNTQxMmMyZDcwMmE3NTNiNjExZTYxODc='}
{'snapshot_id': 'NzY4LGY5ZmU2NTM4OWQ0M2Q5NWE2NGY3ZDc3MDVkMzE1OThlOWYxZmZhNTY='}
{'snapshot_id': 'NzY5LDllN2Q4MTQ0ODI3MTAzOTQyNjQ4ZmQxZDZlOTA4MWVkNjI2MmE1YzI='}
{'snapshot_id': 'NzcwLDgyZDc2Nzk5NTQyMGY4ZmRhMzE5MGI0ZGVhMzM2OTE3NWVmZDMyYTc='}
{'snapshot_id': 'NzcxLDFjNWU2YjI2NTcwZmI0MWM2MzllNzk0ZDU2N2Q2MWQ0NDc1YmYyMjU='}
{'snapshot_id': 'NzcyLGI0ZjBjOTk2NjE1Njh

# List Playlists

In [11]:
playlists = spot.user_playlists( USER_NAME )
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = spot.next( playlists )
    else:
        playlists = None

   1 spotify:playlist:56Cj4qTNnaeBfci6VYUYw6 Brains and Bastards
   2 spotify:playlist:5TtKaKCouyJp7Hhtu4YlYm zb Approved 1
   3 spotify:playlist:6bopUaWmBByVaImpXRWDlW za Probation 1
   4 spotify:playlist:2PzycpJ8zDQQAcWcjz2RmX Pop 1
   5 spotify:playlist:1VegeaSQSrp5Edu4IVPyo0 Aesop Rock & Other Rap
   6 spotify:playlist:5HmNkICbTDCEwEXUJPdTss zc _ Study Overflow
   7 spotify:playlist:41sFSisljvBDMBXtpp5NIw Study Playlist 04
   8 spotify:playlist:3o3lN2qntdEV7UKTuuC77K Study Playlist 03
   9 spotify:playlist:65MXR4dubPL9t0P4dgTWvn Study Playlist 11
  10 spotify:playlist:6gbtR2cBq5PvkghidCvvGk Study Playlist 02
  11 spotify:playlist:7K9ucByFRgDuZk8KMHeJkL zb _ Study Overflow
  12 spotify:playlist:0ecSAfnD4CulIVnLt26ukI za _ Study Overflow
  13 spotify:playlist:0a2qoe6S7lYeZ6nlhZdA0v Study Playlist 01
  14 spotify:playlist:3832xeKGEOAXFJqE4K8kIq Study Playlist 10
  15 spotify:playlist:5mHRBFoQtYy2izeZ66pG95 Study Playlist 09
  16 spotify:playlist:0tspdJlwSgiyf2O9PO6QaP Study Playlist 0